# CibNav : gestion des données

## Packages et importation des fichiers

In [2]:
import numpy as np
import pandas as pd

In [3]:
folder="C:\\Users\\quentin.mascart\\Documents\\Stage DAM\\CibNav\\"

In [4]:
Navire=pd.read_csv(folder+"Données\\Navire.csv")
Historique=pd.read_csv(folder+"Données\\Dataset_train.csv")

## Gestions des données

On selectionnes les colonnes utiles à notre apprentissage.

In [5]:
Navire=Navire[['ID Nav Flotteur','Date Francisation', 'Annee Construction', 'Genre Navigation', 'Jauge Oslo', 'Longueur Hors Tout', 'Materiau Coque', 'Nombre Moteur', 'Num Version', 'Puissance Administrative', 'Situation Flotteur', 'Type Carburant', 'Type Moteur']]

In [6]:
Historique=Historique[['ID Nav Flotteur','Date Visite','Annee Visite','Sitrep History','Nombre Prescriptions','Nombre Prescriptions Majeurs','Sitrep History']]

In [7]:
df=pd.merge(Historique,Navire,on='ID Nav Flotteur')

On ajoute les colones suivantes :
- age_francisation à la visite
- age à la visite

In [8]:
df['Annee_Francisation']=df['Date Francisation'].str.slice(0, 4)
df['Annee_Francisation']=pd.to_numeric(df['Annee_Francisation'])
df['age_francisation']=df['Annee Visite']-df['Annee_Francisation']
df['age']=df['Annee Visite']-df['Annee Construction']

Ici on supprime les lignes ayant au moins une information manquante. 
Voir comment faire pour gérer les colones manquantes dans les modèles.

In [9]:
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19968 entries, 0 to 27718
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID Nav Flotteur               19968 non-null  float64
 1   Date Visite                   19968 non-null  object 
 2   Annee Visite                  19968 non-null  int64  
 3   Sitrep History                19968 non-null  int64  
 4   Nombre Prescriptions          19968 non-null  float64
 5   Nombre Prescriptions Majeurs  19968 non-null  float64
 6   Sitrep History                19968 non-null  int64  
 7   Date Francisation             19968 non-null  object 
 8   Annee Construction            19968 non-null  float64
 9   Genre Navigation              19968 non-null  object 
 10  Jauge Oslo                    19968 non-null  float64
 11  Longueur Hors Tout            19968 non-null  float64
 12  Materiau Coque                19968 non-null  object 
 13  N

## Pretraitement des données

Recodage des variables catégorielle. Une colonne par catégorie

In [17]:
categories = ['Genre Navigation', 'Materiau Coque', 'Situation Flotteur', 'Type Carburant', 'Type Moteur']
numerique = ['ID Nav Flotteur','Date Visite', 'Jauge Oslo', 'Longueur Hors Tout', 'Nombre Moteur', 'Num Version', 'Puissance Administrative','Nombre Prescriptions','Nombre Prescriptions Majeurs','Sitrep History','age','age_francisation']
cat=df[categories]
num=df[numerique]

In [31]:
Data=pd.read_csv(folder+"Données\\dataframe.csv")

#Création de la cible au moins une prescription majeure

Data['Cible'] = Data['Nombre Prescriptions Majeurs'].apply(lambda x: '1' if x > 0 else '0').astype(int)
#Création de la cible au moins une préscription majeure ou au moins 4 mineurs
Data['Cible_deux'] = Data['Cible'] + Data['Nombre Prescriptions'].apply(lambda x: '1' if x > 3 else '0').astype(int)
Data['Cible_deux'] = Data['Cible_deux'].apply(lambda x: '1' if x > 0 else '0').astype(int)

del Data['Unnamed: 0']

In [32]:
test_size=0.33
df=Data.copy()
df=df.sort_values(by='Date Visite')
del df["Date Visite"]
int(len(Data)*test_size)
X_train=df.head(int(len(Data)*(1-test_size))).sample(frac=1)
X_test=df.tail(int(len(Data)*(test_size))).sample(frac=1)

Y_train=X_train['Cible_deux']
Y_test=X_test['Cible_deux']

del X_train['Cible']
del X_test['Cible']
del X_train['Cible_deux']
del X_test['Cible_deux']

In [35]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_train)

enc.transform(X_train).toarray()

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [30]:
from sklearn.feature_extraction import DictVectorizer
prep = DictVectorizer()
cat_as_dicts = [dict(r.iteritems()) for _, r in cat.iterrows()]
temp = prep.fit_transform(cat_as_dicts)
cat_exp = temp.toarray()
prep.feature_names_

['Genre Navigation=C.M.PetitePêche',
 'Genre Navigation=Cabotage Internat.',
 'Genre Navigation=Cabotage National',
 'Genre Navigation=Conchyl.PetitePêche',
 'Genre Navigation=Conchyl.Pure',
 'Genre Navigation=Lamanage',
 'Genre Navigation=Lamanage Nav cot',
 'Genre Navigation=Navigation côtière',
 'Genre Navigation=Petite pêche',
 'Genre Navigation=Pilotage',
 'Genre Navigation=Plaisance Cabot. Nat',
 'Genre Navigation=Plaisance Cabot.Int.',
 'Genre Navigation=Plaisance Long Cours',
 'Genre Navigation=Plaisance NC',
 'Genre Navigation=Pêche au large',
 'Genre Navigation=Pêche côtière',
 'Genre Navigation=Remorquage CI',
 'Genre Navigation=Remorquage CN',
 'Genre Navigation=Remorquage Nav.Cot.',
 'Materiau Coque=ACIER',
 'Materiau Coque=ACIER / INOX',
 'Materiau Coque=AG/4 - ALU',
 'Materiau Coque=ALLIAGE LEGER',
 'Materiau Coque=AUTRES',
 'Materiau Coque=BOIS',
 'Materiau Coque=BOIS MASSIF',
 'Materiau Coque=BOIS MOULE',
 'Materiau Coque=CONTREPLAQUE',
 'Materiau Coque=METAL',
 'Mater

In [13]:
cat_exp_df = pd.DataFrame( cat_exp, columns = prep.feature_names_ )
reject = ['Situation Flotteur=Français Commerce','Genre Navigation=Remorquage Nav.Cot.','Materiau Coque=AUTRES', 'Type Carburant=AUTRE', 'Type Moteur=Alternative à vapeur']
keep = [ c for c in cat_exp_df.columns if c not in reject ]
cat_exp_df_nocor = cat_exp_df [ keep ]
X = pd.concat ( [num, cat_exp_df_nocor ], axis= 1)
X.shape

(23926, 54)

In [14]:
X=X.dropna()

In [15]:
X.columns

Index(['ID Nav Flotteur', 'Date Visite', 'Jauge Oslo', 'Longueur Hors Tout',
       'Nombre Moteur', 'Num Version', 'Puissance Administrative',
       'Nombre Prescriptions', 'Nombre Prescriptions Majeurs',
       'Sitrep History', 'Sitrep History', 'age', 'age_francisation',
       'Genre Navigation=C.M.PetitePêche',
       'Genre Navigation=Cabotage Internat.',
       'Genre Navigation=Cabotage National',
       'Genre Navigation=Conchyl.PetitePêche', 'Genre Navigation=Conchyl.Pure',
       'Genre Navigation=Lamanage', 'Genre Navigation=Lamanage Nav cot',
       'Genre Navigation=Navigation côtière', 'Genre Navigation=Petite pêche',
       'Genre Navigation=Pilotage', 'Genre Navigation=Plaisance Cabot. Nat',
       'Genre Navigation=Plaisance Cabot.Int.',
       'Genre Navigation=Plaisance Long Cours',
       'Genre Navigation=Plaisance NC', 'Genre Navigation=Pêche au large',
       'Genre Navigation=Pêche côtière', 'Genre Navigation=Remorquage CI',
       'Genre Navigation=Remorquag

In [16]:
X.to_csv('dataframe.csv')